## 🎨 Monet like Painting 👩🏻‍🎨 with stable diffusion model 👨🏻‍💻

For more detailed overview, please checkout our [github](https://github.com/mohammadzainabbas/Deep-Learning-CS/) repo.

In [1]:
from os.path import join, exists, isfile, isdir, abspath, dirname, basename, realpath
from os import makedirs, listdir, pardir, getcwd, chdir

from dataclasses import dataclass, field
from typing import Union, List, Dict
import pandas as pd
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
import logging as log
from enum import Enum

from IPython.display import display, Javascript

from dash import Dash, dcc, html, Input, Output
from dash.exceptions import PreventUpdate

import json

from jupyter_dash import JupyterDash
import plotly.express as px

import plotly.graph_objects as go

import plotly.tools as tls
from plotly.offline import plot, iplot

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

#### Default configurations

In [2]:
#--------------------------
# some configuration(s)
#--------------------------
chdir("/Users/mohammadzainabbas/Masters/CS/Machine-Learning/Deep-Learning-CS")
data_dir = join(getcwd(), "data")
with open(join(data_dir, "captions.json"), 'r') as f: 
   captions_dict = json.load(f)
   captions = [{"image": k, "caption": v} for k, v in captions_dict.items()]
    
# data_files = [x for x in listdir(data_dir) if isfile(join(data_dir, x))]
# algos = [x.name for x in algorithms_to_run]
# sample_size = 50
# show_algo_comparsion=True
# show_dataset_comparsion=False
# show_verbose=False

[{'image': '00068bc07f.jpg',
  'caption': 'a grassy field with a mountain range'},
 {'image': '000910d219.jpg',
  'caption': 'a large body of water with mountains'},
 {'image': '000ded5c41.jpg',
  'caption': 'two people standing on a beach near the ocean'},
 {'image': '00104fd531.jpg',
  'caption': 'a cloudy sky with a single cloud in the sky'},
 {'image': '001158d595.jpg',
  'caption': 'a large body of water with a lighthouse'},
 {'image': '0033c5f971.jpg',
  'caption': 'a brown and white bird is standing in the grass'},
 {'image': '0039ebb598.jpg', 'caption': 'a view from the top of a mountain'},
 {'image': '003aab6fdd.jpg',
  'caption': 'a brick building with a clock on the side of it'},
 {'image': '003c6c30e0.jpg',
  'caption': 'a grassy hillside overlooking a body of water'},
 {'image': '00479e2a21.jpg',
  'caption': 'two boats are docked in the water at sunset'},
 {'image': '005f987f56.jpg',
  'caption': 'a sailboat in the middle of a large body of water'},
 {'image': '0080f94ebc

### Demo

In [3]:
#------------------------------------
# Interactive component(s)
#------------------------------------
external_stylesheets = ['https://fonts.googleapis.com/css?family=Roboto:300,400,500,700&display=swap', 'https://fonts.googleapis.com/icon?family=Material+Icons']
app = JupyterDash(__name__, external_stylesheets=external_stylesheets)

In [4]:
app.layout = html.Div([
    html.Button('Click here to see the content', id='show-secret'),
    html.Div(id='body-div')
])

@app.callback(
    Output(component_id='body-div', component_property='children'),
    Input(component_id='show-secret', component_property='n_clicks')
)
def update_output(n_clicks):
    if n_clicks is None:
        raise PreventUpdate
    else:
        return "Elephants are the only animal that can't jump"

In [7]:
app.run_server(mode='inline')

In [ ]:
w_dataset = html.Div([
    html.Label("Dataset"),
    dcc.Dropdown(
        id="w_dataset",
        options=[{"label": x, "value": x} for x in data_files],
        value=data_files[0],
        clearable=False
    )
], style={"padding": "10px"})

w_algo_list = html.Div([
    html.Label("Algorihtm(s)"),
    dcc.Dropdown(
        id="w_algo_list",
        options=[{"label": convert_to_normal(x), "value": x} for x in algos],
        value=algos[0],
        multi=True,
    )
], style={"padding": "10px"})

w_sample_size = html.Div([
    html.Label("Sample size"),
    dcc.Input(
        id="w_sample_size",
        type="number",
        value=sample_size,
        min=1,
        max=1000,
        step=1
    )
], style={"padding": "10px"})

w_show_algo_comparsion = html.Div([
    html.Label("Show algorithm comparsion"),
    dcc.Checklist(
        id="w_show_algo_comparsion",
        options=[{"label": " ", "value": "show"}],
        value=["show"] if show_algo_comparsion else [],
        labelStyle={"display": "inline-block"}
    )
], style={"padding": "10px"})

w_show_dataset_comparsion = html.Div([
    html.Label("Show dataset comparsion"),
    dcc.Checklist(
        id="w_show_dataset_comparsion",
        options=[{"label": " ", "value": "show"}],
        value=["show"] if show_dataset_comparsion else [],
        labelStyle={"display": "inline-block"}
    )
], style={"padding": "10px"})

w_show_verbose = html.Div([
    html.Label("Show verbose"),
    dcc.Checklist(
        id="w_show_verbose",
        options=[{"label": " ", "value": "show"}],
        value=["show"] if show_verbose else [],
        labelStyle={"display": "inline-block"}
    )
], style={"padding": "10px"})

app.layout = html.Div([
    html.H1("Centrality measures"),
    html.Div([
        w_dataset,
        w_algo_list,
        w_sample_size,
        # w_show_algo_comparsion,
        # w_show_dataset_comparsion,
        # w_show_verbose
    ], style={"padding": "16px"}),
    html.Div([
        html.Div(id="w_output")
    ], style={})
], style={"background": "white","padding": "16px","display": "flex","flex-direction": "column"})

In [ ]:
@app.callback(
    Output("w_output", "children"),
    [
        Input("w_dataset", "value"),
        Input("w_algo_list", "value"),
        Input("w_sample_size", "value"),
        # Input("w_show_algo_comparsion", "value"),
        # Input("w_show_dataset_comparsion", "value"),
        # Input("w_show_verbose", "value")
    ]
)
# def update_output(dataset, algo_list, sample_size, show_algo_comparsion, show_dataset_comparsion, show_verbose):
def update_output(dataset, algo_list, sample_size):
    
    
    data_file = join(data_dir, dataset)
    show_verbose = False
    
    graph = nx.read_edgelist(data_file, nodetype=str, data=(("weight", float),))
    graph = sample_graph(graph=graph, sample_size=sample_size, verbose=show_verbose, plot=False)
    di_graph = undirected_to_directed(graph)
    
    algorithms = [x for x in algorithms_to_run if x.name in algo_list]
    centrality_measures = compute_algorithms(graph=graph, algorithms=algorithms, verbose=show_verbose)

    html_output = []

    for i, centrality_measure in enumerate(centrality_measures):
        html_output.append(
            dcc.Graph(
                figure=plot_network_graph(graph=graph, centrality_result=centrality_measure),
                style={"width": "100%", "height": "100%"},
                id=f"graph_{i}_{centrality_measure.name}"
            )
        )

    return html.Div(html_output)

In [ ]:
app.run_server(mode='inline')